# Syntethic Data

## Generation

In [ ]:
!pip install langchain_openai==0.2.12

In [ ]:
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG_Refined.csv

--2024-12-15 23:05:19--  https://raw.githubusercontent.com/MasrourTawfik/Textra_Insights/main/Files/PCG_Refined.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120894 (118K) [text/plain]
Saving to: ‘PCG_Refined.csv’

PCG_Refined.csv     100%[===================>] 118.06K  --.-KB/s    in 0.02s   

2024-12-15 23:05:19 (6.96 MB/s) - ‘PCG_Refined.csv’ saved [120894/120894]



In [ ]:
PCG_Refined = pd.read_csv("PCG_Refined.csv")
PCG_Refined.head()

,Id,Title,Definition
0,2111,Frais de constitution,Les frais de constitution sont les dépenses en...
1,2112,Frais préalables au démarrage,Les frais préalables au démarrage sont les dép...
2,2113,Frais d'augmentation du capital,Les frais d'augmentation du capital sont les c...
3,2114,"Frais sur opérations de fusions, scissions et ...","Les frais sur opérations de fusions, scissions..."
4,2116,Frais de prospection,Les frais de prospection sont les coûts engagé...


In [ ]:
print("Number of accounts :",len(PCG_Refined["Id"].unique()))

Number of accounts : 83


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import time
from tqdm import tqdm
import random
import pandas as pd
import json

GITHUB_TOKEN = "xxxxxxxxxxxxxxxxxxxx-xxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
class SyntheticData(BaseModel):
    generated_scenarios: str = Field("", description="Les 30 nouveaux scénarios générés en format json")



System_Prompt = """
### Système Prompt

Tu es un expert en comptabilité marocaine et en application du **Plan Comptable Marocain (PCM)**. Ton rôle est de générer 30 scénarios réalistes et concis pour chaque définition de compte que je te fournirai. Ces scénarios doivent décrire des transactions spécifiques effectuées par une entreprise.

**Exigences pour chaque scénario :**
- Chaque scénario doit être précis, réaliste et s'appliquer à des entreprises marocaines ou des environnements similaires.
- Donner des 30 descriptions un petit peut longues .
- Respecte les règles et l’esprit du Plan Comptable Marocain.
- Les réponses doivent être formatées en **JSON** avec cette structure :

 "id": 1,
 "prompt": "Scénario réaliste 1"

 "id": 2,
 "prompt": "Scénario réaliste 2"

Les valeurs pour `id` doivent être numériques croissantes, et `prompt` doit contenir la description concise du scénario.
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", System_Prompt),
        ("human", """
       La definition depuis le Plan Comptable Marocain est la suivante :
        {Definition}
        """),
    ]
)

LLM = ChatOpenAI(model="gpt-4o-mini",
    #temperature=0,
    api_key=GITHUB_TOKEN,
    base_url="https://models.inference.ai.azure.com",
)

LLM_Structered = LLM.with_structured_output(SyntheticData)
LLM_Generator = prompt | LLM_Structered

In [ ]:
def Run_Generator(definition):
  Input = {"Definition": definition}
  response = LLM_Generator.invoke(Input)
  return response

def Run_And_Parse(AccountID, PCG_Refined):
    Definition = PCG_Refined[PCG_Refined["Id"] == AccountID]["Definition"].values[0]

    df = pd.DataFrame(columns=["anchor", "positive", "negative"])

    for i in range(3):
        print(f"Iteration {i+1}")
        try:
            response = Run_Generator(Definition)
            parsed_json = json.loads(response.generated_scenarios)
            break
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(1)
            continue

    for item in parsed_json:
        anchor_prompt = item["prompt"]
        positive_definition = Definition
        negative_definitions = PCG_Refined[PCG_Refined["Id"] != AccountID]["Definition"].tolist()
        negative_definition = random.choice(negative_definitions)

        while negative_definition == Definition:
            negative_definition = random.choice(negative_definitions)

        new_row = pd.DataFrame([{
            "anchor": anchor_prompt,
            "positive": positive_definition,
            "negative": negative_definition
        }])

        df = pd.concat([df, new_row], ignore_index=True)

    return df

def Run_ALL(PCG_Refined):
    Final = pd.DataFrame(columns=["anchor", "positive", "negative"])
    counter = 0
    for account_id in tqdm(PCG_Refined["Id"].unique(), desc="Generating Data..."):
      try:
        result_df = Run_And_Parse(account_id, PCG_Refined)
        Final = pd.concat([Final, result_df], ignore_index=True)
        if counter == 10:
          print("Sleeping for 60 seconds...")
          time.sleep(60)
          counter = 0
          # Save your csv in case a crush occured you will not start from the bigening again
          print("Saving CSV...")
          Final.to_csv("Final.csv", index=False)
        else :
          counter += 1
          time.sleep(2)

      except Exception:
        continue

    Final.to_csv("Final.csv", index=False)
    return Final

## Test Code with a definition

In [ ]:
example = PCG_Refined.iloc[53]["Definition"]
response = Run_Generator(example)
parsed_json = json.loads(response.generated_scenarios)

print(example)
print(json.dumps(parsed_json, indent=4, ensure_ascii=False))

Ce compte enregistre les paiements effectués pour des services d'intermédiaires et d'honoraires. Cela inclut les commissions versées à des agents commerciaux, les honoraires d'avocats ou d'experts-comptables, ainsi que les frais liés aux actes juridiques et aux contentieux.

1. Une entreprise paie une commission de 5% à un agent immobilier pour la vente d'un bien. 2. Un avocat reçoit un honoraire de 150 euros pour une consultation juridique. 3. Un expert-comptable facture 1 000 euros pour la préparation des états financiers d'une entreprise. 4. Un notaire est rémunéré pour 300 euros pour la rédaction d'un acte de vente. 5. Une société paie des frais de 200 euros à un huissier pour l'exécution d'un jugement. 6. Un consultant reçoit 800 euros pour un audit de l'entreprise. 7. Une agence de publicité perçoit une commission de 10% sur les ventes générées par une campagne. 8. Un courtier en assurance est rémunéré pour une prime de 2 500 euros. 9. Un architecte facture 5 000 euros pour la co

In [ ]:
Run_ALL(PCG_Refined)

Generating Data...:   0%|          | 0/84 [00:00<?, ?it/s]

Iteration 1


Generating Data...:   1%|          | 1/84 [00:16<22:56, 16.58s/it]

Iteration 1


Generating Data...:   2%|▏         | 2/84 [00:25<16:37, 12.17s/it]

...

Saving CSV...
Iteration 1


Generating Data...:  99%|█████████▉| 83/84 [38:14<00:36, 36.36s/it]

Iteration 1


Generating Data...: 100%|██████████| 84/84 [38:26<00:00, 27.45s/it]


,anchor,positive,negative
0,Une entreprise de services a engagé un avocat ...,Les frais de constitution sont les dépenses li...,Les avances et acomptes sur immobilisations co...
1,Un entrepreneur a dû payer un notaire 3 000 di...,Les frais de constitution sont les dépenses li...,Les pertes sur créances irrécouvrables représe...
2,"Pour répondre aux exigences légales, une socié...",Les frais de constitution sont les dépenses li...,Les immobilisations corporelles en cours compr...
3,L'immatriculation de l'entreprise au registre ...,Les frais de constitution sont les dépenses li...,Les terrains bâtis désignent les parcelles de ...
4,Un expert-comptable a été engagé pour établir ...,Les frais de constitution sont les dépenses li...,Les impôts et taxes directs sont des prélèveme...
...,...,...,...
1668,Un café doit évaluer la valeur de ses stocks a...,Les dotations d'exploitation aux provisions po...,Les ouvrages d'infrastructure sont des constru...
1669,Un distributeur de produits cosmétiques consta...,Les dotations d'exploitation aux provisions po...,Les frais d'acquisition des immobilisations so...
1670,Une entreprise de nettoyage doit évaluer ses c...,Les dotations d'exploitation aux provisions po...,Les redevances de crédit-bail sont les paiemen...
1671,Un vendeur de voitures d'occasion doit ajuster...,Les dotations d'exploitation aux provisions po...,Les charges sociales désignent les contributio...


## Push your data to HuggingFace

In [ ]:
!pip install datasets

In [ ]:
Final = pd.read_csv("Final.csv")
Final.shape

(1643, 3)

- You need a HuggingFace acces token, to push your data .
  - Go to your Huggingface profile
  - Settings
  - Acces Tokens
  - Create a new token
  - Write
  - Copy the generated token

In [ ]:
from datasets import Dataset, DatasetDict
from huggingface_hub import HfFolder


repo_id = "username/Reponame"
auth_token = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
#To shuffle the rows of the Final DataFrame before splitting

Final = Final.sample(frac=1, random_state=42).reset_index(drop=True)

test_size = 0.2
val_size = 0.1

test_split = int(len(Final) * test_size)
val_split = int(len(Final) * val_size)

train_df = Final.iloc[:-test_split-val_split]
test_df = Final.iloc[-test_split:]
val_df = Final.iloc[-test_split-val_split:-test_split]

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df),
    "validation": Dataset.from_pandas(val_df)
})

dataset.push_to_hub(repo_id, token=auth_token)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Noureddinesa/Invoices_embedding_3/commit/16dc23eadb0daa82573a6dc1a2c4321fa9bc727e', commit_message='Upload dataset', commit_description='', oid='16dc23eadb0daa82573a6dc1a2c4321fa9bc727e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Noureddinesa/Invoices_embedding_3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Noureddinesa/Invoices_embedding_3'), pr_revision=None, pr_num=None)

# Install Necessary Packages

In [1]:
!pip install transformers[torch]

In [2]:
!pip install -U sentence-transformers

In [3]:
!pip install datasets

In [4]:
#https://sbert.net/docs/sentence_transformer/training_overview.html
#https://huggingface.co/BAAI/bge-large-en


In [5]:
from sentence_transformers import SentenceTransformer
import torch


#   Load Dataset

In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("username/datasetname")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
test_dataset = dataset["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


More infromation about the dataset over here: # https://sbert.net/docs/sentence_transformer/training_overview.html#dataset


In [7]:
train_dataset

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 1151
})

In [8]:
train_dataset.to_pandas()
# Anchor: The original sentence or query.
# Positive answer: A correct or relevant response to the anchor.
# Negative answer: An incorrect or irrelevant response to the anchor

,anchor,positive,negative
0,Une collectivité locale verse un acompte de 1 ...,Les avances et acomptes sur immobilisations co...,Les achats de matières et fournitures consomma...
1,Une société de sécurité engage un service de t...,Les transports regroupent les frais liés au dé...,Les redevances de crédit-bail sont les paiemen...
2,"Lors de l'importation de boissons gazeuses, l'...",Les impôts et taxes indirects sont des prélève...,Le mobilier de bureau désigne l'ensemble des m...
3,Une entreprise technologique développe un prot...,Les immobilisations corporelles en cours compr...,"Le compte ""6141. Etudes, recherches et documen..."
4,Une entreprise de construction à Essaouira res...,Les charges sociales désignent les contributio...,L'achat de marchandises du groupe B désigne l'...
...,...,...,...
1146,Un terrain pour un centre culturel a été aména...,Les terrains aménagés représentent des parcell...,Les services bancaires désignent les frais ass...
1147,Une société de distribution lance un projet de...,"Le compte ""6141. Etudes, recherches et documen...",Les entretiens et réparations désignent les dé...
1148,Un cabinet d'architecture verse une avance de ...,Les avances et acomptes sur immobilisations co...,Les frais d'acquisition des immobilisations so...
1149,Un hôtel doit payer des taxes sur les spectacl...,"Les impôts, taxes et droits assimilés sont des...","Les frais liés aux opérations de fusion, sciss..."


# Load Model

In [9]:
# 1. Load a model to finetune with 2. (Optional) model card data

#popular embedding models:
#https://huggingface.co/nomic-ai/nomic-embed-text-v1
#https://huggingface.co/BAAI/bge-large-en

model = SentenceTransformer("Alibaba-NLP/gte-multilingual-base",trust_remote_code=True)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Setting up Training Arguments

In [10]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers



In [11]:
# 3. Define a loss function
loss = MultipleNegativesRankingLoss(model)


In [12]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/mpnet-base-all-nli-triplet",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=100,
    logging_steps=50,
    run_name="mpnet-base-all-nli-triplet",  # Will be used in W&B if `wandb` is installed
)

# Train

In [13]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)


In [14]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: monojob111 (monojob111-jjbnnmn). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
50,0.466500,0.122677
100,0.140700,0.099875
150,0.083900,0.065370
200,0.060700,0.092077
250,0.038300,0.060587
300,0.056700,0.046577
350,0.018000,0.054437
400,0.014300,0.050594
450,0.003700,0.049877
500,0.004400,0.059414


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=864, training_loss=0.051894362208528305, metrics={'train_runtime': 1557.2924, 'train_samples_per_second': 2.217, 'train_steps_per_second': 0.555, 'total_flos': 0.0, 'train_loss': 0.051894362208528305, 'epoch': 3.0})

# Test - Model any good?

In [ ]:
from sentence_transformers.evaluation import TripletEvaluator

test_evaluator = TripletEvaluator(
    anchors=test_dataset["anchor"],
    positives=test_dataset["positive"],
    negatives=test_dataset["negative"],
    name="all-nli-test",
)

test_evaluator(model)


{'all-nli-test_cosine_accuracy': 1.0}

In [15]:

# 8. Save the trained model
model.save_pretrained("models/mpnet-base-all-nli-triplet/checkpoint-864")
auth_token = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
# 9. (Optional) Push it to the Hugging Face Hub
model.push_to_hub("Model_Name",token=auth_token)

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/Noureddinesa/Invoices_gte-multilingual-base/commit/b0b85eb8671baa926cb4fa2b96e6727655da6629'